In [1]:
#Finbert model
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [52]:
#Test the model
from transformers import pipeline

classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)
result = classifier("Nuclear war has broken out between Russia and the USA", top_k=None)

result = result

print(result[0]['score'])

print(result)

Device set to use mps:0


0.7090891003608704
[{'label': 'negative', 'score': 0.7090891003608704}, {'label': 'neutral', 'score': 0.25406184792518616}, {'label': 'positive', 'score': 0.03684897720813751}]


In [29]:
#test of fetching news data from the news api
from pygooglenews import GoogleNews
import json
import time 
import pandas as pd

gn = GoogleNews()
search = gn.search('^omx', when='3650d') 
news_data = search['entries']

news_data = pd.DataFrame(news_data)
news_data.to_csv("news_data.csv")

#we want to take only the headlines and the date from the news_data

news_titles = news_data['title']

#Save titles to a file
news_titles.to_csv("news_titles.csv")


In [30]:
#we want to load news_titles and enumerate them
import pandas as pd
import numpy as np

news_titles = pd.read_csv('news_titles.csv')

news_titles.drop('Unnamed: 0', axis=1, inplace=True)

#Show all the titles
news_titles



,title
0,European Market Activity - Nasdaq
1,Orbea Orca OMX Custom SRAM Red AXS Build Revie...
2,Early Data for OMX-0407 Fuel Expansion Cohorts...
3,New Drug Could Sustain Oxygen-Starved Hearts -...
4,Orbea Oiz OMX: First Ride Review - Bikers Edge
...,...
95,Finland stocks lower at close of trade; OMX He...
96,Nasdaq readjusts in OMX Stockholm Benchmark In...
97,Knight Composites goes off-road with new MTB w...
98,Finland stocks higher at close of trade; OMX H...


In [27]:
#Check the titles with FInbert model
scores = []
for title in news_titles['title']:
    result = classifier(title)
    scores.append(result)
    
#combine the scores with the titles
scores = pd.DataFrame(scores)
news_titles['scores'] = scores
news_titles.to_csv("news_titles_with_scores.csv")


In [38]:
print(classifier("The stock market is going up as fed raises interest rates"))

[{'label': 'positive', 'score': 0.8155226111412048}]


In [39]:
#Sectors relevant to OMX30 
#Sectors: Industrials, Financials, Telecom, Consumer Discretionary 
#Crawl news data for these sectors in regards to Sweden and OMX30

gn = GoogleNews()
search_1 = gn.search('Industrials Sweden', when='3650d')
search_2 = gn.search('Financials Sweden', when='3650d')
search_3 = gn.search('Telecom Sweden', when='3650d')
search_4 = gn.search('Consumer Discretionary Sweden', when='3650d')

#Combine the data into one dataframe
news_data = search_1['entries'] + search_2['entries'] + search_3['entries'] + search_4['entries']
news_data = pd.DataFrame(news_data)

#peforme sentiment analysis on the news data
news_titles = news_data['title']
scores = []
for title in news_titles:
    result = classifier(title)
    scores.append(result)
    
#combine the scores with the titles putting the scores first
scores = pd.DataFrame(scores)
news_data['scores'] = scores
news_data.to_csv("news_data_with_scores_sectors.csv")
